# Qwen2.5-VL and Qwen3

Source: https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct

In [ ]:
!pip install transformers qwen_vl_utils

In [1]:
# from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from transformers import Qwen3VLMoeForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from os import listdir 
from os.path import join 
from PIL import Image
from tqdm import tqdm
from pprint import pprint
import cv2
from pathlib import PureWindowsPath, Path
import pandas as pd

In [33]:
# default: Load the model on the available device(s)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
# )

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="cuda:0",
)

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)

NameError: name 'Qwen2_5_VLForConditionalGeneration' is not defined

In [2]:
# model = Qwen3VLMoeForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen3-VL-30B-A3B-Instruct", dtype="auto", device_map="auto"
# )
model = Qwen3VLMoeForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-30B-A3B-Instruct",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-30B-A3B-Instruct")

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this video."},
        ],
    }
]

# Preparation for inference
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)

# inputs = {k:v.cuda() for k,v in inputs.items()}

In [ ]:
type(inputs.to('cuda'))

In [ ]:


# Inference: Generation of the output
generated_ids = model.generate(**inputs.to('cuda'), max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

Application to one image.

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "file://data/scan.jpg",
            },
            {"type": "text", "text": "Transcribe the handwritten text in this picture."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
content_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    contents=content_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

Application to a batch of images.

In [ ]:
# Handwritten Text Recognition

# content_folder = "data/insect"
# system_prompt = ("You are a classification model. "
#                 "You only gives two possible outputs: "
#                 "'Yes' if the input content contains an insect visiting the "
#                 "flower. 'No' if the input image does not contain an insect "
#                 "or if the insect in the image is not on the flower. "
#                 "Be careful: say 'No' if the insect is not ON the flower "
#                 "but next to it. Say 'Yes' if the insect is hiding behind the flower. Some insect are small and hides.")

content_folder = "data/20250613_htr_2"
note_taker = """
Transcribe handwritten text from scanned pages with precision and fidelity.

Rules:
- Preserve paragraph breaks, indentation, headers, footnotes, and notes.
- Keep line breaks and spacing, unless a line break interrupts a sentence—then continue the line.
- Copy punctuation, spelling, and capitalization exactly, even if archaic or nonstandard.
- If a circled number appears (e.g. ①, ②), always replace it with plain text in this format: (1), (2), etc. Never output circled or Unicode numbers.
- Preserve list structures and numbering as written, adjusting to (1), (2)... if needed.
- Use plain text only—no markdown, symbols, or styling.
- Mark unclear areas as [illegible], [unclear], or [guess].
- Omit crossed-out text.
- Use LaTeX-style for math (e.g., x_{{y}}).
- Ignore non-handwritten content like logos or form lines.
- Do not rewrite or correct the original—transcribe exactly what is written.
"""

reformatting_system="""
Reformat raw notes into clean plain text.

Rules:
- Standardize all numbered items to one consistent format: (1), (2), etc.
- Replace circled numbers (e.g. ①) with (1), (2), etc. Keep them in order.
- Do not use markdown or Unicode symbols. No bullets or asterisks.
- Merge broken lines when a sentence clearly continues.
- Keep paragraph breaks where the topic or comment shifts.
- Retain all original wording. Do not rewrite or correct content.
- Preserve math and variable notation as-is (e.g. θ^rpn, λ₀ = 1).
- Output plain text only. No markdown syntax or special formatting.
"""

note_taker_md = """
You transcribe handwritten notes into structured Markdown format with high fidelity.

Rules:
- Preserve all original content exactly, including spelling, punctuation, and spacing, even if nonstandard or archaic.
- Use Markdown headings for section titles if clearly indicated (e.g. underlined or larger handwriting).
- Use Markdown numbered lists for itemized or circled numbers (① → 1. etc.), keeping the order accurate.
- For inline math or equations, use LaTeX formatting wrapped in `$...$` or `$$...$$` if on its own line.
- Mark illegible areas as **[illegible]** or **[unclear]**. Use **[guess]** for best-guess transcriptions.
- Ignore crossed-out or struck-through text.
- Do not include printed background elements (e.g. lines, logos, templates).
- Do not wrap anything in backticks or asterisks unless clearly present in the handwriting.
- Avoid interpreting or rephrasing — transcribe faithfully.

"""

reformatting_system_md="""
Reformat transcribed Markdown notes into clean, structured Markdown for clarity and consistency.

Rules:
- Ensure all numbered items use standard Markdown format (`1.`, `2.`, etc.).
- Standardize any mixed formats (e.g. circled numbers, `(1)`, or inconsistent numbering) into clean Markdown lists.
- Use headings (`##`, `###`, etc.) to organize sections logically based on topic shifts or section titles.
- Merge broken lines within sentences; preserve breaks between ideas, questions, or sections.
- Keep all math in LaTeX syntax (e.g. `$x_{{y}}$`), and preserve variable notation.
- Bold or italicize for clarity only if meaning is implied (e.g. emphasis or notes), but do not invent formatting.
- Mark illegible or unclear text clearly as `[illegible]`, `[unclear]`, or `[guess]`.
- Output must remain plain Markdown — no HTML, no rendered output, no backtick blocks unless content is code.
"""

# Mice detector
content_folder = "data/test/mice/images"
system_prompt="""
You are a wildlife ecologist specialized in rodent and small mammal identification from images.

Your task is to analyze the image and determine if it contains an animal.
If an animal is present, classify it into one of the predefined categories.

Output strictly one of the following labels as a plain string (no explanation, no formatting):

labels = [
    "no_animal",
    "mouse",
    "vole",
    "mustelid",
    "shrew",
    "rat",
    "hedgehog",
    "bird",
    "ungulate",
    "other_animal"
]

If uncertain, choose the most likely category.
Output nothing except the label.
"""
content_type="image"

# Wildlife camera detector
# content_folder = "/home/george/codes/pyplayground/data/test/wildcam/Klelund_0113_211011_211124"
# content_folder = "/home/george/codes/pyplayground/data/test/wildcam/Klelund_0244_220530_220616"
# content_folder = "/home/george/codes/pyplayground/data/test/wildcam/Tofte_0052_230524_230531"
content_folder = "/work/datasets/wildcams/Tofte_0052_230524_230531"
# content_folder = "/work/datasets/wildcams/Klelund_0113_211011_211124"
# content_folder = "/work/datasets/wildcams/Klelund_0244_220530_220616"
# system_prompt_v1="""
# You are an automated wildlife observation analysis system. Your task is to analyze the provided video and extract key information into a single, specific CSV-formatted line.

# **Instructions:**
# 1.  Analyze the entire video for animal presence, behavior, and environmental context.
# 2.  Extract the requested metadata from the text overlay.
# 3.  Your entire output must be a single line of text containing 6 comma-separated values in the exact order specified below.
# 4.  Do not output headers, explanations, or any text other than the single CSV line.
# 5.  If a value cannot be determined, use the specified default.

# **CSV Field Order and Content:**
# 1.  **`animal_present`**: `TRUE` if an animal is visible, otherwise `FALSE`.
# 2.  **`species_common_name`**: Common name. Default: `N/A`.
# 3.  **`species_scientific_name`**: Scientific name. Default: `N/A`.
# 4.  **`individual_count`**: Integer count of the primary species. Default: `0`.
# 5.  **`demographic_composition`**: `Female`, `Male`, `Juvenile` or `Unknown`. Default: `N/A`.
# 6.  **`behavior_description`**: Short, neutral keywords (e.g., `walking, foraging, observing`). Default: `N/A`.
# """
# system_prompt_v2="""
# You are an expert wildlife analyst. Your task is to analyze the provided video and generate a single line of CSV data describing the primary animal species present.

# Use the following header and rules for each field:
# `species_common_name,species_scientific_name,individual_count,demographic_composition,behavior_description`

# 1.  `species_common_name`: The common name of the primary species (e.g., `red deer`).
# 2.  `species_scientific_name`: The scientific name of the primary species. One of: 
# 3.  `individual_count`: The total number of individuals of that species.
# 4.  `demographic_composition`: Describe the primary subject or group. Use one: `male`, `female`, `juvenile`, `mixed group`, `unknown`.
# 5.  `behavior_description`: Provide 1-3 comma-separated keywords describing the behavior (e.g., `"walking,observing"`).

# If no animal is visible in the video, output the following default values:
# `N/A,N/A,0,N/A,N/A`

# Here is an example of valid output:
# red deer,cervus elaphus,1,female,"walking,observing"
# """
# system_prompt="""
# You are an expert wildlife analyst. Your task is to analyze the provided video and generate a single line of CSV data describing the primary animal species present.

# Use the following header and rules for each field:
# `species_common_name,individual_count,demographic_composition`

# 1.  `species_common_name`: The common name of the primary species (e.g., `red deer`). One of: hare, red deer, roe deer, fox, boar, badger, wolf, cat, raccoon dog, marten sp., martes martes, mustelid sp., otter, polecat, squirrel, martes foina, fallow deer, hedgehog, carnivore sp., bird, unidentified mammal, other agents (explain in comments), dog, vehicle, ungulate sp.
# 2.  `individual_count`: The total number of individuals of that species.
# 3.  `demographic_composition`: Describe the primary subject or group. Use one: `male`, `female`, `juvenile`, `mixed group`, `unknown`.

# If no animal is visible in the video, output the following default values:
# `N/A,0,N/A`

# Here is an example of valid output:
# red deer,1,female
# """
system_prompt="""
You are an expert wildlife analyst. Your task is to analyze the provided video and generate a single line of CSV data describing the primary animal species present. 

Use the following header and rules for each field:
`species_common_name,individual_count,demographic_composition`

1.  `species_common_name`: The common name of the primary species (e.g., `red deer`). If a human or a vehicle is present in the video, put 'human' or 'vehicle'
2.  `individual_count`: The total number of individuals of that species.
3.  `demographic_composition`: Describe the primary subject or group. Use one: `male`, `female`, `juvenile`, `mixed group`, `unknown`.

If no animal and no vehicle and no human is visible in the video, output the following default values:
`N/A,0,N/A`

Here is an example of valid output:
red deer,1,female
"""
content_type="video"
# header="species_common_name,species_scientific_name,individual_count,demographic_composition,behavior_description"
header="filename,species_common_name,individual_count,demographic_composition"
output_filename="video_analysis.csv"
add_content_path = True

# Target detector in wildlife camera videos
# content_folder = "/home/george/codes/pyplayground/data/test/wildcam/Klelund_0113_211011_211124"
# # content_folder = "/home/george/codes/pyplayground/data/test/wildcam/Klelund_0244_220530_220616"
# # content_folder = "/home/george/codes/pyplayground/data/test/wildcam/Tofte_0052_230524_230531"
# system_prompt="""
# You are a wildlife video classifier. Analyze the video and respond with a single word based on these rules:

# *   `target`: If any non-human animal is present.
# *   `empty`: If no animals, humans, or vehicles are present.
# *   `human`: If any human or vehicle is present.
# """
# content_type="video"
# header="filename,animal_present"
# output_filename="is_there_a_target.csv"
# add_content_path = True

In [ ]:
messages = [
    [{
        "role": "system",
        "content": system_prompt,
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": f"file://{join(content_folder, p)}"}
        ],
    }] for p in sorted(listdir(content_folder))
]

# Preparation for inference
texts = [
    processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
    for msg in messages
]
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=texts,
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Batch Inference
generated_ids = model.generate(**inputs, max_new_tokens=1024)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_texts = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_texts)

In [ ]:
# Adapt a bit the note taker
# system_prompt = note_taker

output_texts = []

# Sort image paths
content_paths = sorted(listdir(content_folder))

def is_image_or_video(file_path):
    # Check if it's an image
    try:
        with Image.open(file_path) as img:
            img.verify()
        return True
    except:
        pass

    # Check if it's a video
    try:
        cap = cv2.VideoCapture(file_path)
        if cap.isOpened():
            cap.release()
            return True
    except:
        pass

    return False

# Process images one at a time
for content_name in content_paths:
    content_path = join(content_folder, content_name)
    if not is_image_or_video(content_path):
        print(f"Path {content_path} is not an image.")
        continue

    # Construct message
    messages = [
        # {
        #     "role": "system",
        #     "content": system_prompt,
        # },
        {
            "role": "user",
            "content": [
                {"type": content_type, content_type: f"file://{content_path}"},
                {"type": "text", "text": system_prompt},
            ],
        },
    ]

    # Format for model input
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info([messages])

    inputs = processor(
        text=text,
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference
    generated_ids = model.generate(**inputs, max_new_tokens=1024)
    generated_ids_trimmed = generated_ids[0][len(inputs.input_ids[0]) :]  # Single sample
    output_text = processor.decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    if add_content_path:
        output_text = content_path + "," + output_text
    output_texts.append(output_text)

print("Transcription done. Uniformization...")

# # Last pass to uniformized the text
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "text", "text": f"Rewrite the text by following the rules:\n\nText:\n{output_texts}\n\nRules:\n{system_prompt}"},
#         ],
#     }
# ]

# # Preparation for inference
# text = processor.apply_chat_template(
#     messages, tokenize=False, add_generation_prompt=True
# )
# inputs = processor(
#     text=[text],
#     padding=True,
#     return_tensors="pt",
# )
# inputs = inputs.to("cuda")

# # Inference: Generation of the output
# generated_ids = model.generate(**inputs, max_new_tokens=2048)
# generated_ids_trimmed = [
#     out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
# ]
# output_text = processor.batch_decode(
#     generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
# )

# print("Done uniformizing. Saving...")


# Done
transcribtion_path = join(content_folder, output_filename)

with open(transcribtion_path, "w", encoding="utf-8") as f:
    if header is not None and len(header) > 0:
        f.write(header+"\n")
    for text in output_texts:
        f.write(text + "\n")

print(f"Transcribtion stored in {transcribtion_path}.")

In [ ]:
transcribtion_path = join(image_folder, "transcriptions.txt")
with open(transcribtion_path, "w", encoding="utf-8") as f:
    for path, text in zip(image_paths, output_texts):
        f.write(path + "," + text + "\n")

print(f"Transcribtion stored in {transcribtion_path}.")

In [ ]:
messages = [
    {
            "role": "system",
            "content": reformatting_system,
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": f"{output_texts}"},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
inputs = processor(
    text=[text],
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=2048)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [ ]:
print(output_text[0])

Application to videos

In [ ]:
# Wildlife camera detector
video_folder = "/home/george/codes/pyplayground/data/test/wildcam/images"
system_prompt="""
You are an automated wildlife observation analysis system. Your task is to analyze the provided video and extract key information into a single, specific CSV-formatted line.

**Instructions:**
1.  Analyze the entire video for animal presence, behavior, and environmental context.
2.  Extract all metadata from the text overlay.
3.  Your entire output must be a single line of text containing 13 comma-separated values in the exact order specified below.
4.  Do not output headers, explanations, or any text other than the single CSV line.
5.  If a value cannot be determined, use the specified default.

**CSV Field Order and Content:**
1.  **`video_filename`**: Use the placeholder `[VIDEO_FILENAME]`.
2.  **`animal_present`**: `TRUE` if an animal is visible, otherwise `FALSE`.
3.  **`species_common_name`**: Common name. Default: `N/A`.
4.  **`species_scientific_name`**: Scientific name. Default: `N/A`.
5.  **`individual_count`**: Integer count of the primary species. Default: `0`.
6.  **`sex`**: `Female`, `Male`, or `Unknown`. Default: `N/A`.
7.  **`behavior_description`**: Short, neutral keywords (e.g., `walking, foraging, observing`). Default: `N/A`.
8.  **`habitat_description`**: Short, neutral keywords (e.g., `pine forest, grassland, riverbed`).
9.  **`camera_id`**: Alphanumeric ID from the overlay.
10. **`timestamp_utc`**: Full timestamp from overlay, converted to `YYYY-MM-DDTHH:MM:SSZ` format.
11. **`temperature_celsius`**: Numeric temperature value.
12. **`pressure`**: Numeric pressure value.
"""

In [ ]:
messages = [
    [{
        "role": "system",
        "content": system_prompt,
    },
    {
        "role": "user",
        "content": [
            {"type": "video", "video": f"file://{join(video_folder, p)}"}
        ],
    }] for p in sorted(listdir(video_folder))
]

# Preparation for inference
texts = [
    processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
    for msg in messages
]
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=texts,
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Batch Inference
generated_ids = model.generate(**inputs, max_new_tokens=1024)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_texts = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_texts)

Try to design my own chatting bot

In [ ]:
from transformers import TextIteratorStreamer
from qwen_vl_utils import process_vision_info
import os
from threading import Thread

# --- The Main Chat Interface Class for API Usage ---
class QwenVLChat:
    """
    A versatile chat class for Qwen2.5-VL that supports both blocking (API-style)
    and streaming (generator-style) responses.
    """

    def __init__(self, model, processor, system_prompt=None):
        self.model = model
        self.processor = processor
        self.history = []
        self.system_msg=None
        if system_prompt is not None and isinstance(system_prompt, str):
            self.system_msg={"role": "system", "content": system_prompt}
        self.device = model.device
        print("QwenVLChat initialized. Ready to chat.")

    def _prepare_inputs_history(self, prompt: str = None, image_path: str = None):
        """Internal method to prepare model inputs and update history."""
        # Eventually add system prompt
        msg = []
        if self.system_msg is not None:
            msg.append(self.system_msg)

        # Create the user message content
        content = []
        if image_path: # TODO: adapt to URLs
            if not os.path.exists(image_path):
                print(f"Warning: Image path not found: {image_path}. Ignoring image.")
            else:
                content.append({"type": "image", "image": f"file://{image_path}"})
        if prompt is not None:
            content.append({"type": "text", "text": prompt})

        # Append the new user message to the conversation history
        msg.append({"role": "user", "content": content})

        # Append to the history
        self.history.append(msg)

        # Process the entire history for the model
        texts = [self.processor.apply_chat_template(
            msg, tokenize=False, add_generation_prompt=True
        ) for msg in self.history]
        image_inputs, video_inputs = process_vision_info(self.history)
        
        inputs = self.processor(
            text=texts,
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        ).to(self.device)
        
        return inputs
    
    def _prepare_inputs(self, prompt: str = None, image_path: str = None):
        """Internal method to prepare model inputs and update history."""
        # Eventually add system prompt
        msg = []
        if self.system_msg is not None:
            msg.append(self.system_msg)

        # Create the user message content
        content = []
        if image_path: # TODO: adapt to URLs
            if not os.path.exists(image_path):
                print(f"Warning: Image path not found: {image_path}. Ignoring image.")
            else:
                content.append({"type": "image", "image": f"file://{image_path}"})
        if prompt is not None:
            content.append({"type": "text", "text": prompt})

        # Append the new user message to the conversation history
        msg.append({"role": "user", "content": content})

        # Process the entire history for the model
        texts = [self.processor.apply_chat_template(
            msg, tokenize=False, add_generation_prompt=True
        )]
        image_inputs, video_inputs = process_vision_info(msg)
        
        inputs = self.processor(
            text=texts,
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        ).to(self.device)
        
        return inputs

    def generate_response(self, prompt: str = None, image_path: str = None, generation_args: dict = None) -> str:
        """
        Generates a complete, blocking response. This is the most robust method
        and handles all cases, including image-only inputs.
        It is not limited by token count and generates until the model stops.
        
        Args:
            prompt (str, optional): The text prompt.
            image_path (str, optional): The path to the image file.
            generation_args (dict, optional): Override generation parameters.
        
        Returns:
            str: The full text response from the model.
        """
        print(self.history)
        inputs = self._prepare_inputs(prompt, image_path)

        # Set a high max_new_tokens to allow for long responses, mimicking "unlimited" generation.
        # The model will stop on its own when it generates an EOS token.
        gen_kwargs = {
            "max_new_tokens": 4096,
            "do_sample": True,
            "temperature": 0.7,
            "top_p": 0.9,
            **(generation_args or {})
        }

        # Generate the token IDs
        generated_ids = self.model.generate(**inputs, **gen_kwargs)
        # generated_ids = self.model.generate(**inputs, max_new_tokens=128)

        # Trim the input tokens from the generated output
        generated_ids_trimmed = [
            out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]

        # Decode the trimmed token IDs to get the final text
        # We access [0] because batch_decode returns a list
        response_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )
        
        # Now that generation is successful, update the history
        self.history.append({"role": "assistant", "content": response_text})
        
        return response_text

    def chat(self, prompt: str = None, image_path: str = None, generation_args: dict = None) -> str:
        """
        A simple, blocking chat method.

        Args:
            prompt (str): The text prompt from the user.
            image_path (str, optional): The path to the image file.
            generation_args (dict, optional): Arguments for model.generate().

        Returns:
            str: The complete response from the model.
        """
        # Simply call the streaming method and concatenate the results.
        assert prompt is not None or image_path is not None, "Please provide either a prompt or an image path."
        response_tokens = []
        for token in self.chat_stream(prompt, image_path, generation_args):
            response_tokens.append(token)
            
        return "".join(response_tokens)

    def chat_stream(self, prompt: str = None, image_path: str = None, generation_args: dict = None):
        """
        A streaming chat method that yields tokens as they are generated.

        Args:
            prompt (str): The text prompt from the user.
            image_path (str, optional): The path to the image file.
            generation_args (dict, optional): Arguments for model.generate().

        Yields:
            str: The next token generated by the model.
        """
        assert prompt is not None or image_path is not None, "Please provide either a prompt or an image path."
        inputs = self._prepare_inputs(prompt, image_path)
        streamer = TextIteratorStreamer(self.processor.tokenizer, skip_prompt=True, skip_special_tokens=True)

        # Default generation arguments
        gen_args = {
            "max_new_tokens": 2048,
            "do_sample": True,
            "temperature": 0.7,
            "top_p": 0.9,
            **inputs,
            "streamer": streamer,
        }
        if generation_args:
            gen_args.update(generation_args)
        
        # Run generation in a separate thread to not block the main thread
        thread = Thread(target=self.model.generate, kwargs=gen_args)
        thread.start()

        # Yield tokens as they become available and build the full response
        full_response = []
        for new_token in streamer:
            yield new_token
            full_response.append(new_token)
        
        thread.join() # Ensure the generation thread is finished

        # After streaming is complete, add the full assistant response to history
        self.history.append({"role": "assistant", "content": "".join(full_response)})

    def get_history(self):
        """Returns the current conversation history."""
        return self.history

    def clear_history(self):
        """Resets the conversation history."""
        self.history = []
        print("Conversation history cleared.")


In [ ]:
# --- Initialize Chatbot ---
chatbot = QwenVLChat(model, processor)

# --- Example API Interaction ---
image_file = "data/scan.jpg" # IMPORTANT: Make sure this file exists
if not os.path.exists(image_file):
    print(f"\nERROR: The image '{image_file}' was not found.")
    print("Please download or create a sample image and place it in the 'data' directory.")
    exit()

# --- DEMO 1: Using the simple, blocking `chat()` API ---
print("\n--- DEMO 1: Simple Blocking API Call ---")
prompt1 = "Transcribe the handwritten text in this picture."
print(f"User: {prompt1}")

# The call blocks until the full response is ready
response1 = chatbot.chat(prompt=prompt1, image_path=image_file)

print(f"Assistant: {response1}")
print("----------------------------------------\n")

# --- DEMO 2: Using the `chat_stream()` API for real-time output ---
print("--- DEMO 2: Streaming API Call ---")
prompt2 = "Based on the text you transcribed, please write a short summary."
print(f"User: {prompt2}")
print("Assistant (streaming): ", end="", flush=True)

# The call returns a generator immediately. We loop through it.
full_response_streamed = ""
for token in chatbot.chat_stream(prompt=prompt2):
    print(token, end="", flush=True)
    full_response_streamed += token
print("\n----------------------------------------\n")

# --- DEMO 3: Verify conversation history ---
print("--- DEMO 3: Verifying Conversation History ---")
print("The conversation history is maintained across both blocking and streaming calls.")

# A final blocking call that relies on the context from the previous turns
prompt3 = "What was the very first thing I asked you to do?"
print(f"User: {prompt3}")
response3 = chatbot.chat(prompt=prompt3)
print(f"Assistant: {response3}")
print("----------------------------------------\n")

In [ ]:
system_prompt = ("You are a classification model. "
                "You only gives two possible outputs: "
                "'Yes' if the input image contains an insect visiting the "
                "flower. 'No' if the input image does not contain an insect "
                "or if the insect in the image is not on the flower.")
insect_detector = QwenVLChat(model, processor, system_prompt)

In [ ]:
image_file = "data/insect/WSCT9000.JPG"
response1 = insect_detector.generate_response(image_path=image_file)

print(f"Assistant: {response1}")
print("----------------------------------------\n")

OOM below:

In [ ]:
from transformers import pipeline
import torch

pipe = pipeline("image-text-to-text", model="Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="cuda:0", use_fast=True)
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    },
]
pipe(text=messages)

# InternVL3

Source: https://huggingface.co/OpenGVLab/InternVL3-8B

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
path = "OpenGVLab/InternVL3-8B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()


In [ ]:
import math
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

def split_model(model_name, world_size=None):
    device_map = {}
    world_size = torch.cuda.device_count() if world_size is None else world_size
    config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers
    # Since the first GPU will be used for ViT, treat it as half a GPU.
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.model.rotary_emb'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

In [ ]:
# set the max number of tiles in `max_num`
pixel_values = load_image('data/what.jpg', max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=1024, do_sample=True)

# pure-text conversation (纯文本对话)
question = 'Hello, who are you?'
response, history = model.chat(tokenizer, None, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

In [ ]:
question = '<image>\nPlease describe the image shortly.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')

In [ ]:
pixel_values = load_image('data/scan.jpg', max_num=12).to(torch.bfloat16).cuda()
question = '<image>\nPlease transcribe the handwritten text in this picture.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')

## Qwen3 on wildcams

In [4]:
vid_dir = Path("/work/datasets/wildcams/testData")
csv_pth = Path("/work/datasets/wildcams/WildCam_testDataSet.csv")
trg_pth = Path("/work/datasets/wildcams/deepfaune_1.4_ids.csv")

In [5]:
to_linux_format = lambda  x : x.str.replace(r'\\', '/', regex=True)

df=pd.read_csv(csv_pth)
df['filepath'] = to_linux_format(df['filepath'])
df_trg=pd.read_csv(trg_pth)
df_trg['filename'] = to_linux_format(df_trg['filename'])

In [6]:
possible_outputs = list(df_trg['prediction'].unique())
possible_types = list(df['type'].unique())

In [7]:
vid_pth=vid_dir / Path(*Path(df['filepath'].iloc[0]).parts[2:])

In [8]:
for i,r in df.iterrows():
    vid_pth=vid_dir / Path(*Path(df['filepath'].iloc[0]).parts[2:])

,filename,filepath,video_name,area,series,TimeStampCET,year,date,time,species,count,type,certainty
0,Klelund_0001_200907_200911_file_20200909_040300,D:/NHMA/LUDA/Klelund_0001/Klelund_0001_200907_...,Rec_20200909_020311_151_M,Klelund,Klelund_0001_200907_200911,2020-09-09 04:03:22,2020,2020-09-09,04:03:00,fox,1,adult (unsp. Sex),Helt sikker
1,Klelund_0001_200907_200911_file_20200910_123500,D:/NHMA/LUDA/Klelund_0001/Klelund_0001_200907_...,Rec_20200910_103531_151_M,Klelund,Klelund_0001_200907_200911,2020-09-10 12:35:42,2020,2020-09-10,12:35:00,red deer,1,female adult,Helt sikker
2,Klelund_0002_200907_200928_file_20200912_183600,D:/NHMA/LUDA/Klelund_0002/Klelund_0002_200907_...,Rec_20200912_163616_151_M,Klelund,Klelund_0002_200907_200928,2020-09-12 18:36:29,2020,2020-09-12,18:36:00,roe deer,1,male adult,Helt sikker
3,Klelund_0002_200907_200928_file_20200913_014700,D:/NHMA/LUDA/Klelund_0002/Klelund_0002_200907_...,Rec_20200912_234706_151_M,Klelund,Klelund_0002_200907_200928,2020-09-13 01:47:17,2020,2020-09-13,01:47:00,fox,1,adult (unsp. Sex),Mulig/overvejende sandsynlig
4,Klelund_0002_200907_200928_file_20200916_021000,D:/NHMA/LUDA/Klelund_0002/Klelund_0002_200907_...,Rec_20200916_001019_151_M,Klelund,Klelund_0002_200907_200928,2020-09-16 02:10:30,2020,2020-09-16,02:10:00,fox,1,adult (unsp. Sex),Helt sikker
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8692,Klelund_0279_230824_231005_file_20230826_121900,D:/NHMA/LUDA/Klelund_0279/Klelund_0279_230824_...,Rec_20230826_101921_151_M,Klelund,Klelund_0279_230824_231005,2023-08-26 12:19:37,2023,2023-08-26,12:19:36,roe deer,2,male adult,Helt sikker
8693,Klelund_0279_230824_231005_file_20230826_205200,D:/NHMA/LUDA/Klelund_0279/Klelund_0279_230824_...,Rec_20230826_185236_151_M,Klelund,Klelund_0279_230824_231005,2023-08-26 20:52:48,2023,2023-08-26,20:52:48,roe deer,1,male adult,Helt sikker
8694,Klelund_0279_230824_231005_file_20230909_023500,D:/NHMA/LUDA/Klelund_0279/Klelund_0279_230824_...,Rec_20230909_003529_151_M,Klelund,Klelund_0279_230824_231005,2023-09-09 02:35:42,2023,2023-09-09,02:35:42,red deer,3,female adult,Helt sikker
8695,Klelund_0279_230824_231005_file_20230913_063900,D:/NHMA/LUDA/Klelund_0279/Klelund_0279_230824_...,Rec_20230913_043922_151_M,Klelund,Klelund_0279_230824_231005,2023-09-13 06:39:34,2023,2023-09-13,06:39:34,roe deer,1,female adult,Helt sikker


In [14]:
def is_image_or_video(file_path):
    # Check if it's an image
    try:
        with Image.open(file_path) as img:
            img.verify()
        return True
    except:
        pass

    # Check if it's a video
    try:
        cap = cv2.VideoCapture(file_path)
        if cap.isOpened():
            cap.release()
            return True
    except:
        pass

    return False


system_prompt=f"""
You are an expert wildlife analyst. Your task is to analyze the provided video and generate a single line of CSV data describing the primary animal species present. 

Use the following header and rules for each field:
`prediction,count,type,score`

1.  `prediction`: The common name of the primary species. One of {possible_outputs}
2.  `count`: The total number of individuals of that species.
3.  `type`: Describe the primary subject or group. Use one: {possible_types}.
4.  `score`: An integer between 0 and 99 indicating the level of confidence of the prediction.

Here is an example of valid output:
red deer,1,female,99
"""

def qwen_pred(content_path, system_prompt, content_type="video", add_content_path=True):
    if not is_image_or_video(content_path):
        print(f"Path {content_path} is not an image.")
        return ""

    # Construct message
    messages = [
        # {
        #     "role": "system",
        #     "content": system_prompt,
        # },
        {
            "role": "user",
            "content": [
                # {"type": content_type, content_type: f"file://{content_path}"},
                {"type": content_type, content_type: str(content_path)},
                {"type": "text", "text": system_prompt},
            ],
        },
    ]

    # Format for model input
    # text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    # image_inputs, video_inputs = process_vision_info([messages])

    # inputs = processor(
    #     text=text,
    #     images=image_inputs,
    #     videos=video_inputs,
    #     padding=True,
    #     return_tensors="pt",
    # )
    # inputs = inputs.to("cuda")

    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    )
    inputs = inputs.to("cuda")

    # Inference
    generated_ids = model.generate(**inputs, max_new_tokens=1024)
    generated_ids_trimmed = generated_ids[0][len(inputs.input_ids[0]) :]  # Single sample
    output_text = processor.decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    if add_content_path:
        output_text = str(content_path) + "," + output_text
    return str(output_text)

In [15]:
vid_pth

PosixPath('/work/datasets/wildcams/testData/LUDA/Klelund_0001/Klelund_0001_200907_200911/Rec_20200909_020311_151_M.mp4')

In [17]:
for i,r in df.iterrows():
    vid_pth=vid_dir / Path(*Path(r['filepath']).parts[2:])
    print(qwen_pred(vid_pth, system_prompt))
    if i==10: break

/work/datasets/wildcams/testData/LUDA/Klelund_0001/Klelund_0001_200907_200911/Rec_20200909_020311_151_M.mp4,fox,1,adult (unsp. Sex),95
/work/datasets/wildcams/testData/LUDA/Klelund_0001/Klelund_0001_200907_200911/Rec_20200910_103531_151_M.mp4,roe deer,1,male adult,95
/work/datasets/wildcams/testData/LUDA/Klelund_0002/Klelund_0002_200907_200928/Rec_20200912_163616_151_M.mp4,roe deer,1,adult (unsp. Sex),95
/work/datasets/wildcams/testData/LUDA/Klelund_0002/Klelund_0002_200907_200928/Rec_20200912_234706_151_M.mp4,fox,1,adult (unsp. Sex),85
/work/datasets/wildcams/testData/LUDA/Klelund_0002/Klelund_0002_200907_200928/Rec_20200916_001019_151_M.mp4,empty,0,empty,99
/work/datasets/wildcams/testData/LUDA/Klelund_0002/Klelund_0002_200907_200928/Rec_20200917_222512_151_M.mp4,fox,1,adult (unsp. Sex),95
/work/datasets/wildcams/testData/LUDA/Klelund_0002/Klelund_0002_200907_200928/Rec_20200926_221927_151_M.mp4,fox,1,adult (unsp. Sex),95
/work/datasets/wildcams/testData/LUDA/Klelund_0003/Klelund_000